In [10]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# 1. Pré-processar os dados
# Carregue seus datasets balanceados
train_data = pd.read_csv('../data/preprocessed/movies_genres_grouped_train_preprocessed.csv')
test_data = pd.read_csv('../data/preprocessed/movies_genres_grouped_test_preprocessed.csv')

category_columns = train_data.columns.drop(['Name', 'Description', 'Combined'])

# Defina os parâmetros de pré-processamento
max_features = 10000  # Número máximo de palavras a serem usadas (palavras mais frequentes)
maxlen = 200  # Número máximo de palavras no texto

# Tokenize os textos
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_data['Combined'])

X_train = tokenizer.texts_to_sequences(train_data['Combined'])
X_test = tokenizer.texts_to_sequences(test_data['Combined'])

# Padronize os textos
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Obtenha os labels one-hot encoded
y_train = train_data[category_columns].values
y_test = test_data[category_columns].values

# 2. Construir o modelo
embedding_dim = 128  # Dimensão do vetor de embedding
lstm_units = 64  # Unidades LSTM

model = Sequential([
    Embedding(max_features, embedding_dim, input_length=maxlen),
    Bidirectional(LSTM(lstm_units, return_sequences=True)),
    Bidirectional(LSTM(lstm_units)),
    Dense(len(category_columns), activation='sigmoid')
])

# 3. Treinar o modelo
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

batch_size = 32
epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# 4. Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

Epoch 1/10
125/125 [==============================] - 200s 1s/step - loss: 0.4806 - accuracy: 0.3445 - val_loss: 0.4251 - val_accuracy: 0.4086
Epoch 2/10
125/125 [==============================] - 165s 1s/step - loss: 0.3994 - accuracy: 0.4172 - val_loss: 0.3978 - val_accuracy: 0.4605
Epoch 3/10
125/125 [==============================] - 175s 1s/step - loss: 0.3268 - accuracy: 0.5645 - val_loss: 0.3984 - val_accuracy: 0.4921
Epoch 4/10
125/125 [==============================] - 166s 1s/step - loss: 0.2583 - accuracy: 0.6483 - val_loss: 0.4300 - val_accuracy: 0.4718
Epoch 5/10
125/125 [==============================] - 169s 1s/step - loss: 0.2036 - accuracy: 0.6749 - val_loss: 0.4632 - val_accuracy: 0.4492
Epoch 6/10
125/125 [==============================] - 175s 1s/step - loss: 0.1655 - accuracy: 0.6912 - val_loss: 0.4904 - val_accuracy: 0.4379
Epoch 7/10
125/125 [==============================] - 178s 1s/step - loss: 0.1330 - accuracy: 0.7040 - val_loss: 0.5463 - val_accuracy: 0.4605

In [11]:
from sklearn.metrics import classification_report

# Prever e converter as previsões para o formato binarizado
y_pred = model.predict(X_test)
y_pred_binarized = np.round(y_pred)

# Calcular e exibir métricas de avaliação
report = classification_report(y_test, y_pred_binarized, target_names=category_columns, zero_division=0)
print(report)

41/41 [==============================] - 9s 177ms/step
                         precision    recall  f1-score   support

                 Comedy       0.62      0.50      0.55       502
                  Crime       0.65      0.53      0.58       259
                  Drama       0.73      0.76      0.74       757
                Romance       0.54      0.33      0.41       272
   Action and Adventure       0.64      0.54      0.59       450
Documentary and History       0.47      0.32      0.38       144
   Family and Animation       0.48      0.30      0.37       141
     Fantasy and Sci-Fi       0.52      0.38      0.44       154
    Horror and Thriller       0.51      0.45      0.48       291

              micro avg       0.63      0.53      0.57      2970
              macro avg       0.57      0.46      0.50      2970
           weighted avg       0.62      0.53      0.57      2970
            samples avg       0.65      0.55      0.56      2970



In [12]:
from sklearn.metrics import f1_score, average_precision_score, precision_recall_curve, precision_score, recall_score

y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
precision_micro = precision_score(y_test, y_pred, average='micro')
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_micro = recall_score(y_test, y_pred, average='micro')
recall_macro = recall_score(y_test, y_pred, average='macro')
print("F1-score (micro): {:.2f}".format(f1_micro))
print("F1-score (macro): {:.2f}".format(f1_macro))
print("Precision (micro): {:.2f}".format(precision_micro))
print("Precision (macro): {:.2f}".format(precision_macro))
print("Recall (micro): {:.2f}".format(recall_micro))
print("Recall (macro): {:.2f}".format(recall_macro))

y_pred = model.predict(X_test)
f1_scores_per_class = f1_score(y_test, np.round(y_pred), average=None)
auc_pr_per_class = average_precision_score(y_test, y_pred, average=None)
for i, category in enumerate(category_columns):
    print("Class: {}".format(category))
    print("F1-score: {:.2f}".format(f1_scores_per_class[i]))
    print("AUC-PR: {:.2f}".format(auc_pr_per_class[i]))
    print("\n")

41/41 [==============================] - 9s 222ms/step
F1-score (micro): 0.57
F1-score (macro): 0.50
Precision (micro): 0.63
Precision (macro): 0.57
Recall (micro): 0.53
Recall (macro): 0.46
41/41 [==============================] - 10s 247ms/step
Class: Comedy
F1-score: 0.55
AUC-PR: 0.63


Class: Crime
F1-score: 0.58
AUC-PR: 0.63


Class: Drama
F1-score: 0.74
AUC-PR: 0.77


Class: Romance
F1-score: 0.41
AUC-PR: 0.46


Class: Action and Adventure
F1-score: 0.59
AUC-PR: 0.66


Class: Documentary and History
F1-score: 0.38
AUC-PR: 0.39


Class: Family and Animation
F1-score: 0.37
AUC-PR: 0.39


Class: Fantasy and Sci-Fi
F1-score: 0.44
AUC-PR: 0.39


Class: Horror and Thriller
F1-score: 0.48
AUC-PR: 0.50


